In [ ]:
import pandas as pd
import numpy as np
import glob
from src import *
import src
import os
from PIL import Image

## Generate Labels

In [ ]:
# generate_labels(img_pth_lst=img_pth_lst, lbl_msk_pths = lbl_msk_pths).lbl_masks_to_img_lbls(save_path = r"Z:\__Organized_Directories_InProgress\Annotated_Images\GoPro_FishSegmentation\GoPro_FishSegmentation_FullSize\Dan_Dataset\YOLO_labels", color=None)


In [ ]:
# functions
def get_im_w(image_path):
    im = Image.open(image_path)
    w, h = im.size
    return w
def get_im_h(image_path):
    im = Image.open(image_path)
    w, h = im.size
    return h
lim_w = lambda f: get_im_w(f)
lim_h = lambda f: get_im_h(f)

In [ ]:
def do_train_test_valid_split(df, train_split=0.6, valid_split=0.2):
    l = len(df)
    print("n samples:", l)
    train_df = df.sample(int(train_split*l), random_state=42)
    df = df.drop(train_df.index)
    valid_df = df.sample(int(valid_split*l), random_state=42)
    df = df.drop(valid_df.index)
    test_df = df
    X_train, y_train = train_df.image_path.values, train_df.lbl_path.values
    X_valid, y_valid = valid_df.image_path.values, valid_df.lbl_path.values
    X_test, y_test = test_df.image_path.values, test_df.lbl_path.values
    print(f"training, testing, validation, {X_train.shape[0]}, {X_test.shape[0]},{X_valid.shape[0]}")
    return X_train, y_train, X_valid, y_valid, X_test, y_test, train_df, valid_df, test_df

In [ ]:
X_train, y_train, X_valid, y_valid, X_test, y_test, train_df, valid_df, test_df = do_train_test_valid_split(imgs_lbls_gopro)


In [ ]:
## If you want to save the img/lbl in the same text file
def save_list(x, y, name):
    with open(f"{name}.txt", "w") as f:
        for X, Y in zip(x,y):
            f.write('%s\n' %X)
            f.write('%s\n' %Y)
# save_list(X_train, y_train, "train")
# save_list(X_test, y_test, "test")
# save_list(X_valid, y_valid, "validation")


In [ ]:
## Count objects in set
def count_objects(which_set):
    folder = f"datasets\\datasets_gopro\\{which_set}\\labels\\*txt"
    label_list = glob.glob(folder)
    i = 0
    n = 0
    for label_file in label_list:
        try:
            df = pd.read_csv(label_file, delimiter=' ', header=None)
            num_objects = df.shape[0]
            i += num_objects
        except:
            n += 1
    print(f"num objects in {which_set} is {i}, num images no fish is {n}")
count_objects('train')

In [ ]:
## copy images and labels
import shutil
def cpy_lbls(set, dst):
    for item in set:
        shutil.copy(item, dst)
# cpy_lbls(y_train, r"datasets\datasets_gopro\train\labels")
# cpy_lbls(y_valid, r"datasets\datasets_gopro\validation\labels")
# cpy_lbls(y_test, r"datasets\datasets_gopro\test\labels")
# cpy_lbls(X_train, r"datasets\datasets_gopro\train\images")
# cpy_lbls(X_valid, r"datasets\datasets_gopro\validation\images")
# cpy_lbls(X_test, r"datasets\datasets_gopro\test\images")

In [ ]:
## Plot label overlay
# i = 0
import PIL
from PIL import ImageDraw
i=22 #<---Lots of goby, 6964

print(i)
img_file = X_train[i]
print(img_file)
def display_lbl(X_train, i):
    img_file = X_train[i]
    label = y_train[i]
    df = pd.read_csv(label, delimiter=" ", header=None)
    print(df)
    imgp = PIL.Image.open(img_file)
    im_array = np.array(imgp)
    im_h, im_w = im_array.shape[0], im_array.shape[1]
    draw = PIL.ImageDraw.Draw(imgp)
    s=5
    for index, row in df.iterrows():
        x, y, w, h = row[1]*im_w, row[2]*im_h, row[3]*im_w, row[4]*im_h
        x1 = x - w/2
        y1 = y - h/2
        x2 = x + w/2
        y2 = y + h/2
        draw.rectangle((x1,y1,x2,y2), outline=(0, 0, 0), width=5)
        draw.ellipse((x-s,y-s,x+s,y+s), fill='red')
    return imgp
display_lbl(X_train, i)

# New scores after manual Annotation
1. Run 02_YOLO_predict.py with the appropriate weights and images with corresponding labels
2. Generate a scores sheet with 04_YOLO_calc_scores_df.ipynb
3. Generate annotated images with that scores sheet
4. Create the base annotation template then add the formulas for eas of use
5. After annotation is complete, create the updated scores sheet to compare metrics
6. Calculate the change in metrics


In [ ]:
## 3. Generate annotated images
new_path = r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Train_edit\lbl_imgs"
img_pths = glob.glob(r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\datasets\datasets_gopro\train\images\*.jpg")
scores_df = pd.read_csv(r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Train_edit\yolov8x_GoPro_train_test_scores.csv", index_col=0)
# for img_pth in img_pths:
#     src.generate_labels().save_annot_imgs(img_pth, scores_df, new_path, background="image")

In [ ]:
### 4. Create the base annotation template then add the formulas for eas of use

## Manually added formulas into the resulting excel sheet for better user experience
## CELL G --> =IF(F2="","",IF(F2=D2,"confirmed","changed"))
## CELL H --> =IF(D2=F2,0,IF(D2>F2,0,IF(D2<F2,1)))
## CELL I --> =IF(D2=F2,0,IF(D2<F2,0,IF(D2>F2,1)))

# scores_csv = r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Validation_edit\yolov8x_GoPro_valid_test_scores.csv"
# save_excel = r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Validation_edit\yolov8x_GoPro_valid_test_scores_to_annot.xlsx"

# scores_csv = r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Train_edit\yolov8x_GoPro_train_test_scores.csv"
scores_csv = r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\inference\LMBS_Goby_Photos_conf0.05\LMBS_Goby_Photos_conf0.05_Yolo_predictions.csv"
save_excel = r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\inference\LMBS_Goby_Photos_conf0.05\LMBS_Goby_Photos_conf0.05_Yolo_predictions_QAQC.xlsx"
def scores_to_annot(scores_csv, scored=False):
    scores_df = pd.read_csv(scores_csv, index_col=0)

    if scored:
        df_edit = scores_df[["filename", "detect_id", "tp", "fp"]]
        df_edit = df_edit.sort_values(by="detect_id")
        df_edit["CHECK"] = np.where(df_edit.tp == 1, 1, np.nan)
    else:
         df_edit = scores_df[["filename", "detect_id", "conf"]]
         df_edit = df_edit.copy()
         df_edit["path"] = df_edit.filename.apply(lambda x: os.path.dirname(x.replace("_", "/")))
         df_edit["filename"] = df_edit.filename.apply(lambda x: x.split("_")[-1])
         df_edit["detect_id"] = df_edit.detect_id.apply(lambda x: x.split("_")[-2]+" " + x.split("_")[-1])
        #  df_edit = df_edit.sort_values(by="detect_id")
         df_edit["CHECK"] = np.nan
    return df_edit
scores_to_annot(scores_csv).to_excel(save_excel)


In [ ]:
### 5. After annotation is complete, create the updated scores sheet to compare metrics

scores_annoted_xls = r"Z:\Proj_Yolo\GobyFinder_GoPro_Model_Results\Test_edit\yolov8x_GoPro_test_scores_to_annot.xlsx"
scores_csv = r"test_runs\detect\yolov8x_GoPro_2_results\yolov8x_GoPro_2_scores.csv"
labels_csv = r"test_runs\detect\yolov8x_GoPro_2_results\yolov8x_GoPro_2_Labels.csv"
## Updates scores based on the scores_to_annot.xlsx
def scores_edited(scores_csv, scores_annoted_xls):
    scores_df = pd.read_csv(scores_csv, index_col=0)
    scored_annot_df = pd.read_excel(scores_annoted_xls, index_col=0)
    df_lbls = pd.read_csv(labels_csv, index_col=0)
    n_ground_truth = len(df_lbls)
    scores_edited = pd.merge(scores_df, scored_annot_df[['detect_id', 'fp-tp', 'tp-fp']], on = 'detect_id')
    scores_edited['tp_n'] = scores_edited.tp + scores_edited['fp-tp'] - scores_edited['tp-fp']
    scores_edited['fp_n'] = scores_edited.fp - scores_edited['fp-tp'] + scores_edited['tp-fp']
    scores_edited['acc_tp_n'] = scores_edited.tp_n.cumsum()
    scores_edited['acc_fp_n'] = scores_edited.fp_n.cumsum()
    scores_edited['Precision_n'] = scores_edited.acc_tp_n/(scores_edited.acc_tp_n + scores_edited.acc_fp_n)
    n_ground_truth_n = n_ground_truth + (scores_edited.acc_tp_n.max() - scores_edited.acc_tp.max()) # new ground truth gobies base on annotation
    scores_edited['Recall_n'] = scores_edited.acc_tp_n/n_ground_truth_n
    name = os.path.basename(scores_annoted_xls)
    folder = os.path.dirname(scores_annoted_xls)
    name = str(name).split(".")[0]+"_updated.csv"
    scores_edited_df = scores_edited
    # scores_edited.to_csv(os.path.join(folder, name))
    return n_ground_truth, n_ground_truth_n, scores_edited_df
n_ground_truth, n_ground_truth_n, scores_edited_df = scores_edited(scores_csv, scores_annoted_xls)

In [ ]:
## 6. Calculate the change in metrics

import matplotlib.pyplot as plt
def plot_curves(df, labels=None, gt=0, gt_n=0):
    TP = df.acc_tp.values
    TPn = df.acc_tp_n.values
    FN = gt - TP
    FNn = gt_n - TPn
    FP = df.acc_fp.values
    FPn = df.acc_fp_n.values
    x = df.conf.values
    # m = df.miou.values
    # f = df.F1_score.values
    p = df.Precision
    pi = [p[0]] + [max(p[i+1:]) for i in range(len(p)-1)]
    pn = df.Precision_n
    pin = [pn[0]] + [max(pn[i+1:]) for i in range(len(pn)-1)]
    r = df.Recall
    rn = df.Recall_n
    f1 = 2*(p*r)/(p+r)
    f1n = 2*(pn*rn)/(pn+rn)
    maxf1 = f1.max()
    maxf1n = f1n.max()
    # a = (m+f)/2
    fig, ax = plt.subplots(1, figsize = (6,4))

    # ax.plot(x, r, label='Recall', linestyle='-')
    # ax.plot(x, pi, label='Precision', linestyle='-')
    # ax.plot(x, f1, marker='.', label=f'F1 score {maxf1:0.3f}', markersize=1)

    ax.plot(x, rn, label='Recall_n', linestyle='--')
    ax.plot(x, pin, label='Precision_n', linestyle='--')
    # ax.plot(x, f1, marker='.', label=f'F1 score {maxf1:0.3f}', markersize=1)
    
    ax.set_ylabel('score')
    ax.set_xlabel('Confidence threshold')


    ax.legend()
    ax.grid()
    plt.tight_layout()

    print(f"max F1: {maxf1:0.2f}, num ground truth fish {n_ground_truth}")
    print(f"new max F1: {maxf1n:0.2f}, num ground truth fish {n_ground_truth_n}")
plot_curves(scores_edited_df, gt = n_ground_truth, gt_n = n_ground_truth_n)
